# Episode 1: The Phantom [Table] Menace

This notebook is meant for in-house demonstration of candidate extraction and featurization of tables. It assumes an input file in XHTML format, a strict form of HTML that coincides with XML structure, allowing for easy display (HTML) and safe tree traversal (XML).

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Candidate Extraction

First, import the 'HTMLParser' class to read HTML tables

In [81]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/diseases.xhtml')

The "TableParser" class divides the html doc into cells, adding a 'cell_id' attribute to each cell for future traversal, and creating "Cell" objects that have attributes such as row number, column number, html tag, html attributes, and any tags/attributes on a cells ancestors in the table.

In [94]:
from snorkel.parser import TableParser
table_parser = TableParser()

As usual, pass these to a Corpus object for digestion.

In [95]:
from snorkel.parser import Corpus
%time corpus = Corpus(html_parser, table_parser)

Parsing documents...
Parsing contexts...
Parsed 1 documents and 2 contexts
CPU times: user 63.5 ms, sys: 6.09 ms, total: 69.6 ms
Wall time: 98.5 ms


Load the good 'ole disease dictionary for recognizing disease names.

In [84]:
from load_dictionaries import load_disease_dictionary

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

Loaded 507899 disease phrases!


Here we use a new CandidateSpace object, CellNgrams. It inherits from Ngrams, and ensures that the Table context object is broken up into cells before being passed into the usual routine for pulling out Ngrams.

In [96]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import DictionaryMatch

# Define a candidate space
table_ngrams = TableNgrams(n_max=3)

# Define a matcher
disease_matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Passing the CandidateSpace, Matcher, and Context objects to a Candidates object, extraction is performed, and we see that a number of disease CellNgrams are returned.

In [97]:
from snorkel.candidates import Candidates
%time candidates = Candidates(table_ngrams, disease_matcher, corpus.get_contexts())
for candy in candidates.get_candidates(): print candy

Extracting candidates...
CPU times: user 37.3 ms, sys: 1.88 ms, total: 39.2 ms
Wall time: 38.7 ms
<TableNgram("Scurvy", id=1-0-22-0:0-5, chars=[0,5], (row,col)=(3,1), tag=th)
<TableNgram("Hypochondria", id=1-1-22-0:0-11, chars=[0,11], (row,col)=(3,1), tag=th)
<TableNgram("Problem", id=1-1-1-0:0-6, chars=[0,6], (row,col)=(0,1), tag=th)
<TableNgram("Polio", id=1-0-8-0:0-4, chars=[0,4], (row,col)=(1,1), tag=th)
<TableNgram("Location", id=1-0-3-0:0-7, chars=[0,7], (row,col)=(0,3), tag=th)
<TableNgram("Arthritis", id=1-1-8-0:0-8, chars=[0,8], (row,col)=(1,1), tag=th)
<TableNgram("Disease", id=1-0-1-0:0-6, chars=[0,6], (row,col)=(0,1), tag=th)
<TableNgram("plague", id=1-0-15-1:31-36, chars=[31,36], (row,col)=(2,1), tag=th)
<TableNgram("Fever", id=1-1-15-0:7-11, chars=[7,11], (row,col)=(2,1), tag=th)
<TableNgram("Yellow Fever", id=1-1-15-0:0-11, chars=[0,11], (row,col)=(2,1), tag=th)
<TableNgram("Chicken Pox", id=1-0-15-0:0-10, chars=[0,10], (row,col)=(2,1), tag=th)


### Feature Generation

We can then generate features on our set of candidates, including *new and improved* table features!

In [98]:
from snorkel.features import TableNgramFeaturizer
featurizer = TableNgramFeaturizer(candidates, corpus)

Extracting features...
Extracted 108 features for each of 11 mentions


In [99]:
featurizer.get_features_by_id(candidates.get_candidates()[0].id)

['Ngram_features_to_come',
 'TABLE_ROW_NUM_3',
 'TABLE_COL_NUM_1',
 'TABLE_HTML_TAG_th',
 'TABLE_HTML_ANC_TAG_tr',
 'TABLE_HTML_ANC_TAG_tbody',
 'TABLE_HTML_ANC_TAG_table',
 'TABLE_HTML_ANC_TAG_body',
 'TABLE_HTML_ANC_ATTR_font=blue',
 u'TABLE_ROW_NGRAM_1901',
 u'TABLE_ROW_NGRAM_Scurvy',
 u'TABLE_ROW_NGRAM_Annapolis',
 u'TABLE_COL_NGRAM_Scurvy',
 u'TABLE_COL_NGRAM_Chicken',
 u'TABLE_COL_NGRAM_Chicken_Pox',
 u'TABLE_COL_NGRAM_Chicken_Pox_are',
 u'TABLE_COL_NGRAM_Pox',
 u'TABLE_COL_NGRAM_Pox_are',
 u'TABLE_COL_NGRAM_Pox_are_bad',
 u'TABLE_COL_NGRAM_are',
 u'TABLE_COL_NGRAM_are_bad',
 u'TABLE_COL_NGRAM_are_bad_.',
 u'TABLE_COL_NGRAM_bad',
 u'TABLE_COL_NGRAM_bad_.',
 u'TABLE_COL_NGRAM_.',
 u'TABLE_COL_NGRAM_Disease',
 u'TABLE_COL_NGRAM_So',
 u'TABLE_COL_NGRAM_So_is',
 u'TABLE_COL_NGRAM_So_is_the',
 u'TABLE_COL_NGRAM_is',
 u'TABLE_COL_NGRAM_is_the',
 u'TABLE_COL_NGRAM_is_the_plague',
 u'TABLE_COL_NGRAM_the',
 u'TABLE_COL_NGRAM_the_plague',
 u'TABLE_COL_NGRAM_the_plague_.',
 u'TABLE_COL_NGRA

Ta-da! Next up: feeding this features into the learning machine.